In [1]:
%load_ext autoreload
import os
# os.environ['SNORKELDB'] = 'sqlite:///brat-import.db'
from set_env import set_env
set_env()
from snorkel import SnorkelSession
from snorkel.contrib.brat import Brat
from snorkel.parser import StanfordCoreNLPServer

from snorkel.models import Candidate, Document, candidate_subclass, GoldLabel

session = SnorkelSession()

In [2]:
corenlp_server = StanfordCoreNLPServer(version='3.6.0', split_newline=False, num_threads=1)

In [3]:
brat = Brat(session)

In [4]:
# Only run on documents which have annotations

# from snorkel.parser import TextDocPreprocessor, CorpusParser
# import multiprocessing
# doc_preprocessor = TextDocPreprocessor("brat_data/test_labeled_docs/*.txt", encoding="utf-8")
# corpus_parser = CorpusParser()
# corpus_parser.apply(doc_preprocessor, parralelism=multiprocessing.cpu_count())

Clearing existing...
Running UDF...


In [4]:
input_dir = "brat_data/test_labeled_docs/"
brat.import_project(input_dir, annotations_only=False, annotator_name='brat', num_threads=1, parser=corenlp_server)

CREATED TYPE Entity(Gene,[gene])
CREATED TYPE Entity(Phenotype,[phenotype])
CREATED TYPE Relation(Causation,['gene', 'phenotype'])
CREATED TYPE Relation(Negative,['gene', 'phenotype'])


In [5]:
stable_labels, entity_types = brat.explore()
entity_types

defaultdict(list,
            {'Causation': [[TemporarySpan("AtMND1", sentence=459549, chars=[8,13], words=[2,2]),
               TemporarySpan("did not affect normal vegetative development but caused fragmentation and missegregation of chromosomes in male and female meiosis, formation of inviable gametes, and sterility", sentence=459549, chars=[15,190], words=[3,28])],
              [TemporarySpan("Atmnd1 Atspo11-1", sentence=459551, chars=[16,31], words=[3,4]),
               TemporarySpan("chromosome fragmentation in Atmnd1 was suppressed by loss of Atspo11-1", sentence=459551, chars=[62,131], words=[9,18])],
              [TemporarySpan("s1p-3", sentence=449905, chars=[6,10], words=[2,2]),
               TemporarySpan("sensitive to salt-induced osmotic stress", sentence=449905, chars=[15,54], words=[4,8])],
              [TemporarySpan("zip17", sentence=450055, chars=[38,42], words=[10,10]),
               TemporarySpan("Salt stress sensitivity", sentence=450055, chars=[7,29], word

In [6]:
from snorkel.models import Document, Sentence, Span

In [7]:
PhenoPair = candidate_subclass('Phenotypes', ['descriptor', 'entity'])

In [ ]:
# GenePhenoPair = candiate_subclass('GenePhenoPair', )

In [36]:
import collections
from sqlalchemy import and_

jaccard_cutoff = 0.5

candidates = session.query(PhenoPair).filter(PhenoPair.split == 0).all()
print(len(candidates), "candidates")
sentence_to_pheno = collections.defaultdict(set)

for causation in entity_types['Causation']:
    gene_span, pheno_span = causation
    sentence_to_pheno[pheno_span.sentence.id].add(pheno_span)

num = 0
num_matches = 0
num_sentences = 0
num_gold_phenos = 0
num_zero_matches = 0
avg = 0
for sentence_id, gold_phenos in sentence_to_pheno.iteritems():
    sentence_match = False
    results = session.query(Span, PhenoPair).filter(and_(Span.sentence_id == sentence_id, PhenoPair.descriptor_id == Span.id)).all()
    if (len(results) == 0):
        num_zero_matches += 1
#         print "sentence", sentence_id
#         for gold_pheno in gold_phenos:
#             print(gold_pheno.get_span())
#         print
        
    for gold_pheno in gold_phenos:
        matched = False
        max_match = 0
        for span, cand_pheno in results:
            # Build word indice set for each of the gold phenotype, and the candiate phenotype
            cand_words = set()
            gold_words = set()
            # combine both entities (modifier and attribute) into one "phenotype"
            [cand_words.update(xrange(span.get_word_start(), span.get_word_end()+1)) for span in cand_pheno.get_contexts()]
            
            
            gold_words.update(xrange(gold_pheno.get_word_start(), gold_pheno.get_word_end()+1))
            intersect = gold_words.intersection(cand_words)
            jaccard_score = float(len(intersect)) / len(cand_words.union(gold_words))
            
            if jaccard_score > max_match:
                max_match = jaccard_score
                match_res = (gold_pheno, cand_pheno)
            
            if jaccard_score > jaccard_cutoff:
                matched = True
                sentence_match = True
                avg += jaccard_score
                num += 1
                print(jaccard_score)
                print(gold_pheno)
                print(cand_pheno)
#             print(cand_pheno)
                print("\n")
        num_gold_phenos += 1
        if matched:
            num_matches += 1
    if sentence_match:
        num_sentences += 1

print("avg", avg / num)
recall = float(num_matches) / num_gold_phenos
print("num gold phenos", num_gold_phenos)
num_sentences_tagged = len(sentence_to_pheno.keys())
print("num_sentences_tagged", num_sentences_tagged)
print("sentences", float(num_sentences) / num_sentences_tagged)
print("num sents missed", float(num_zero_matches) / num_sentences_tagged)
print("recall", recall)

(42972, 'candidates')
0.8
TemporarySpan("ratio of shoot weight to primary root length", sentence=380931, chars=[28,71], words=[7,14])
Phenotypes(Span("weight to primary", sentence=380931, chars=[43,59], words=[10,12]), Span("ratio of shoot weight to primary root length of sue4", sentence=380931, chars=[28,79], words=[7,16]))


0.8
TemporarySpan("ratio of shoot weight to primary root length", sentence=380931, chars=[28,71], words=[7,14])
Phenotypes(Span("ratio of", sentence=380931, chars=[28,35], words=[7,8]), Span("ratio of shoot weight to primary root length of sue4", sentence=380931, chars=[28,79], words=[7,16]))


0.8
TemporarySpan("ratio of shoot weight to primary root length", sentence=380931, chars=[28,71], words=[7,14])
Phenotypes(Span("length of", sentence=380931, chars=[66,74], words=[14,15]), Span("ratio of shoot weight to primary root length of sue4", sentence=380931, chars=[28,79], words=[7,16]))


0.727272727273
TemporarySpan("ratio of shoot weight to primary root length",

1.0
TemporarySpan("chlorophyll defects", sentence=403629, chars=[23,41], words=[4,5])
Phenotypes(Span("defects", sentence=403629, chars=[35,41], words=[5,5]), Span("chlorophyll defects", sentence=403629, chars=[23,41], words=[4,5]))


0.615384615385
TemporarySpan("increased the size of the shoot apical region and produced more leaf primordia", sentence=405687, chars=[13,90], words=[2,14])
Phenotypes(Span("increased the size of the", sentence=405687, chars=[13,37], words=[2,6]), Span("size of the shoot apical region", sentence=405687, chars=[27,57], words=[4,9]))


0.615384615385
TemporarySpan("increased the size of the shoot apical region and produced more leaf primordia", sentence=405687, chars=[13,90], words=[2,14])
Phenotypes(Span("increased the size of the", sentence=405687, chars=[13,37], words=[2,6]), Span("more leaf primordia", sentence=405687, chars=[72,90], words=[12,14]))


0.692307692308
TemporarySpan("increased the size of the shoot apical region and produced more leaf prim

TemporarySpan("Total sulphur content in seeds", sentence=381305, chars=[0,29], words=[0,4])
Phenotypes(Span("content in", sentence=381305, chars=[14,23], words=[2,3]), Span("Total sulphur content in seeds of sue3", sentence=381305, chars=[0,37], words=[0,6]))


0.625
TemporarySpan("Total sulphur content in seeds", sentence=381305, chars=[0,29], words=[0,4])
Phenotypes(Span("normal", sentence=381305, chars=[279,284], words=[48,48]), Span("Total sulphur content in seeds of sue3", sentence=381305, chars=[0,37], words=[0,6]))


0.625
TemporarySpan("Total sulphur content in seeds", sentence=381305, chars=[0,29], words=[0,4])
Phenotypes(Span("conditions", sentence=381305, chars=[294,303], words=[50,50]), Span("Total sulphur content in seeds of sue3", sentence=381305, chars=[0,37], words=[0,6]))


0.625
TemporarySpan("Total sulphur content in seeds", sentence=381305, chars=[0,29], words=[0,4])
Phenotypes(Span("free", sentence=381305, chars=[329,332], words=[55,55]), Span("Total sulphur conten



0.571428571429
TemporarySpan("more tolerant to cold stress", sentence=410192, chars=[76,103], words=[13,17])
Phenotypes(Span("was more tolerant to cold", sentence=410192, chars=[72,96], words=[12,16]), Span("pollen", sentence=410192, chars=[43,48], words=[7,7]))


0.588235294118
TemporarySpan("small but significant increase in the number of lateral roots", sentence=377425, chars=[40,100], words=[7,16])
Phenotypes(Span("showed a small but significant increase in the number of lateral", sentence=377425, chars=[31,94], words=[5,15]), Span("significant increase in the number of lateral roots compared to the respective wild-types", sentence=377425, chars=[50,138], words=[9,21]))


0.636363636364
TemporarySpan("greater numbers of first order LRs in unstressed and stressed seedlings but no stimulation of first order LR proliferation in stressed seedlings", sentence=447072, chars=[74,217], words=[12,33])
Phenotypes(Span("greater numbers of first", sentence=447072, chars=[74,97], words=[12,15

0.666666666667
TemporarySpan("young inflorescences and siliques", sentence=402924, chars=[4,36], words=[1,4])
Phenotypes(Span("young inflorescences and", sentence=402924, chars=[4,27], words=[1,3]), Span("siliques of sg1", sentence=402924, chars=[29,43], words=[4,6]))


0.75
TemporarySpan("young inflorescences and siliques", sentence=402924, chars=[4,36], words=[1,4])
Phenotypes(Span("young inflorescences and", sentence=402924, chars=[4,27], words=[1,3]), Span("young inflorescences", sentence=402924, chars=[4,23], words=[1,2]))


0.625
TemporarySpan("sensitive to glucose during germination", sentence=433046, chars=[139,177], words=[18,22])
Phenotypes(Span("developmental arrest are sensitive to", sentence=433046, chars=[114,150], words=[15,19]), Span("sensitive to glucose during germination", sentence=433046, chars=[139,177], words=[18,22]))


0.833333333333
TemporarySpan("sensitive to glucose during germination", sentence=433046, chars=[139,177], words=[18,22])
Phenotypes(Span("insensi

Phenotypes(Span("was only marginally affected in the", sentence=435742, chars=[74,108], words=[12,17]), Span("cortical cell length in the differentiation zone", sentence=435742, chars=[25,72], words=[5,11]))


0.636363636364
TemporarySpan("cortical cell length in the differentiation zone was only marginally affected", sentence=435742, chars=[25,101], words=[5,15])
Phenotypes(Span("length in the differentiation", sentence=435742, chars=[39,67], words=[7,10]), Span("cortical cell length in the differentiation zone", sentence=435742, chars=[25,72], words=[5,11]))


0.6
TemporarySpan("triple response phenotype", sentence=435409, chars=[21,45], words=[4,6])
Phenotypes(Span("triple response", sentence=435409, chars=[21,35], words=[4,5]), Span("triple response phenotype of cul3hyp", sentence=435409, chars=[21,56], words=[4,8]))


0.916666666667
TemporarySpan("no visible root hair elongation in the presence or absence of ACC", sentence=445652, chars=[56,120], words=[10,21])
Phenotypes(Span("el

Phenotypes(Span("contained less", sentence=374137, chars=[60,73], words=[12,13]), Span("less seeds", sentence=374137, chars=[70,79], words=[13,14]))


1.0
TemporarySpan("produced less siliques", sentence=374137, chars=[24,45], words=[4,6])
Phenotypes(Span("produced less", sentence=374137, chars=[24,36], words=[4,5]), Span("less siliques", sentence=374137, chars=[33,45], words=[5,6]))


0.9375
TemporarySpan("significantly less responsive than the wild-type to MeJA-induced root growth inhibition in all light conditions", sentence=445818, chars=[12,122], words=[2,16])
Phenotypes(Span("was significantly less responsive than the wild-type to MeJA-induced", sentence=445818, chars=[8,75], words=[1,9]), Span("significantly less responsive than the wild-type to MeJA-induced root growth inhibition in all light conditions", sentence=445818, chars=[12,122], words=[2,16]))


1.0
TemporarySpan("significantly less responsive than the wild-type to MeJA-induced root growth inhibition in all light condi

0.6
TemporarySpan("reduced auxin levels", sentence=405002, chars=[4,23], words=[1,3])
Phenotypes(Span("reduced", sentence=405002, chars=[4,10], words=[1,1]), Span("reduced auxin levels in adp1-D", sentence=405002, chars=[4,33], words=[1,5]))


0.6
TemporarySpan("protein transport is impaired", sentence=374283, chars=[213,241], words=[37,40])
Phenotypes(Span("impaired in", sentence=374283, chars=[234,244], words=[40,41]), Span("protein transport", sentence=374283, chars=[213,229], words=[37,38]))


0.6
TemporarySpan("early leaf death", sentence=412976, chars=[44,59], words=[7,9])
Phenotypes(Span("early", sentence=412976, chars=[44,48], words=[7,7]), Span("old3-1 early leaf death phenotype", sentence=412976, chars=[37,69], words=[6,10]))


0.625
TemporarySpan("resistance to virulent and avirulent P.syringae is impaired", sentence=439824, chars=[51,109], words=[10,17])
Phenotypes(Span("resistance to virulent and avirulent", sentence=439824, chars=[51,86], words=[10,14]), Span("resistance 

1.0
TemporarySpan("flowers earlier", sentence=425624, chars=[32,46], words=[9,10])
Phenotypes(Span("earlier", sentence=425624, chars=[40,46], words=[10,10]), Span("flowers earlier", sentence=425624, chars=[32,46], words=[9,10]))


0.666666666667
TemporarySpan("flowers earlier", sentence=425624, chars=[32,46], words=[9,10])
Phenotypes(Span("earlier", sentence=425624, chars=[55,61], words=[13,13]), Span("flowers earlier", sentence=425624, chars=[32,46], words=[9,10]))


0.727272727273
TemporarySpan("The amplitudes of the diurnal changes in metabolite levels", sentence=429723, chars=[0,57], words=[0,8])
Phenotypes(Span("amplitudes of the diurnal", sentence=429723, chars=[4,28], words=[1,4]), Span("amplitudes of the diurnal changes in metabolite levels in pgm", sentence=429723, chars=[4,64], words=[1,10]))


0.615384615385
TemporarySpan("The amplitudes of the diurnal changes in metabolite levels", sentence=429723, chars=[0,57], words=[0,8])
Phenotypes(Span("exception of", sentence=429723, 


1.0
TemporarySpan("accelerated first-order rosette branches", sentence=405312, chars=[243,282], words=[35,38])
Phenotypes(Span("branches", sentence=405312, chars=[275,282], words=[38,38]), Span("accelerated first-order rosette branches", sentence=405312, chars=[243,282], words=[35,38]))


1.0
TemporarySpan("ypersensitive to exogenous ABA during germination", sentence=417602, chars=[14,62], words=[5,10])
Phenotypes(Span("germination", sentence=417602, chars=[52,62], words=[10,10]), Span("hypersensitive to exogenous ABA during germination", sentence=417602, chars=[13,62], words=[5,10]))


0.833333333333
TemporarySpan("Proline content was significantly higher", sentence=417606, chars=[4,43], words=[3,7])
Phenotypes(Span("content was significantly higher in", sentence=417606, chars=[12,46], words=[4,8]), Span("Proline content", sentence=417606, chars=[4,18], words=[3,4]))


0.625
TemporarySpan("almost sterile, producing very few seeds", sentence=404449, chars=[18,57], words=[3,9])
Phenoty

In [9]:
# each entity/relation type is assigned to a different split
#for i in range(0,1):

print len(candidates)
for i,c in enumerate(candidates):
    print type(c).type, c
    if i > 5:
        break
print
    
# ProteinComponent relations
candidates = session.query(PhenoPair).filter(PhenoPair.split == 0).all()

42972
Phenotypes.type Phenotypes(Span("First-order rosette branch number was also slightly reduced in", sentence=405564, chars=[0,61], words=[0,8]), Span("also slightly reduced in quadruple mutants compared to wild type", sentence=405564, chars=[38,101], words=[5,14]))
Phenotypes.type Phenotypes(Span("First-order rosette branch number was also slightly reduced in", sentence=405564, chars=[0,61], words=[0,8]), Span("first-order rosette branches", sentence=405564, chars=[120,147], words=[21,23]))
Phenotypes.type Phenotypes(Span("First-order rosette branch number was also slightly reduced in", sentence=405564, chars=[0,61], words=[0,8]), Span("First-order rosette branch number", sentence=405564, chars=[0,32], words=[0,3]))
Phenotypes.type Phenotypes(Span("branches were generated much more slowly than the wild", sentence=405564, chars=[140,193], words=[23,31]), Span("also slightly reduced in quadruple mutants compared to wild type", sentence=405564, chars=[38,101], words=[5,14]))
Phenotype

In [7]:
from snorkel.viewer import SentenceNgramViewer

sv = SentenceNgramViewer(candidates, session=session, n_per_page=6, height=400,
                         annotator_name='brat')
sv

<IPython.core.display.Javascript object>

In [ ]:
# Select spans from brat
# Select candidates from phenos
# For each candidate in phenos
# 